# Finetune Llama-3 with LLaMA Factory

Please use a **free** Tesla T4 Colab GPU to run this!

Project homepage: https://github.com/hiyouga/LLaMA-Factory

## Install Dependencies

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers==0.0.25
!pip install .[bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 9891, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 9891 (delta 99), reused 125 (delta 75), pack-reused 9721
Receiving objects: 100% (9891/9891), 213.57 MiB | 22.40 MiB/s, done.
Resolving deltas: 100% (7276/7276), done.
Updating files: 100% (204/204), done.
/content/LLaMA-Factory
assets/       docker-compose.yml  examples/  pyproject.toml  requirements.txt  src/
CITATION.cff  Dockerfile          LICENSE    README.md       scripts/          tests/
data/         evaluation/         Makefile   README_zh.md    setup.py
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ti8sdzj3/unsloth_7fd625c30a4f41968cc2150eeec90b0f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ti8sdzj3/unsloth_7fd625c30a4f41968cc2150eeec90b0f
  Resolved https://github.com/unslothai/unsloth.git to

### Check GPU environment

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

## Update Identity Dataset

In [ ]:
import json

%cd /content/LLaMA-Factory/

NAME = "Llama-3"
AUTHOR = "LLaMA Factory"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("NAME", NAME).replace("AUTHOR", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)


/content/LLaMA-Factory


## Fine-tune model via LLaMA Board

In [ ]:
from llmtuner import create_ui

%cd /content/LLaMA-Factory/

create_ui().queue().launch(share=True)

## Fine-tune model via Command Line

It takes ~30min for training.

In [ ]:
from llmtuner import run_exp
from llmtuner.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

run_exp(dict(
  stage="sft",                        # do supervised fine-tuning
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset="identity,alpaca_gpt4_en",             # use alpaca and identity datasets
  template="llama3",                     # use llama3 prompt template
  finetuning_type="lora",                   # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  output_dir="llama3_lora",                  # the path to save LoRA adapters
  per_device_train_batch_size=2,               # the batch size
  gradient_accumulation_steps=4,               # the gradient accumulation steps
  lr_scheduler_type="cosine",                 # use cosine learning rate scheduler
  logging_steps=10,                      # log every 10 steps
  warmup_ratio=0.1,                      # use warmup scheduler
  save_steps=1000,                      # save checkpoint every 1000 steps
  learning_rate=5e-5,                     # the learning rate
  num_train_epochs=3.0,                    # the epochs of training
  max_samples=500,                      # use 500 examples in each dataset
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  quantization_bit=4,                     # use 4-bit QLoRA
  loraplus_lr_ratio=16.0,                   # use LoRA+ with lambda=16.0
  use_unsloth=True,                      # use UnslothAI's LoRA optimization for 2x faster training
  fp16=True,                         # use float16 mixed precision training
))

torch_gc()

[INFO|training_args.py:1997] 2024-04-24 18:26:10,599 >> PyTorch: setting up devices
[INFO|training_args.py:1690] 2024-04-24 18:26:10,626 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


/content/LLaMA-Factory
04/24/2024 18:26:10 - WARNING - llmtuner.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.


04/24/2024 18:26:10 - INFO - llmtuner.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16


INFO:llmtuner.hparams.parser:Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
[INFO|tokenization_utils_base.py:2087] 2024-04-24 18:26:12,217 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/tokenizer.json
[INFO|tokenization_utils_base.py:2087] 2024-04-24 18:26:12,219 >> loading file added_tokens.json from 

04/24/2024 18:26:12 - INFO - llmtuner.data.template - Replace eos token: <|eot_id|>


...

04/24/2024 18:26:41 - INFO - llmtuner.train.utils - Using LoRA+ optimizer with loraplus lr ratio 16.00.


INFO:llmtuner.train.utils:Using LoRA+ optimizer with loraplus lr ratio 16.00.
[WARNING|logging.py:329] 2024-04-24 18:26:41,823 >> ==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 591 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 222
 "-____-"     Number of trainable parameters = 20,971,520


Step,Training Loss
10,1.374100
20,1.218500
30,1.029300
40,0.952800
50,0.954000
60,0.980700
70,1.034100
80,0.774500
90,0.770100
100,0.784100


[INFO|<string>:474] 2024-04-24 18:52:42,923 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


[INFO|trainer.py:3305] 2024-04-24 18:52:42,931 >> Saving model checkpoint to llama3_lora
[INFO|configuration_utils.py:726] 2024-04-24 18:52:43,245 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/config.json
[INFO|configuration_utils.py:789] 2024-04-24 18:52:43,248 >> Model config LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  

***** train metrics *****
  epoch                    =        3.0
  total_flos               = 15546268GF
  train_loss               =     0.7403
  train_runtime            = 0:26:01.10
  train_samples_per_second =      1.136
  train_steps_per_second   =      0.142


## Infer the fine-tuned model

In [ ]:
from llmtuner import ChatModel
from llmtuner.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

chat_model = ChatModel(dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="llama3_lora",            # load the saved LoRA adapters
  finetuning_type="lora",                  # same to the one in training
  template="llama3",                     # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
  use_unsloth=True,                     # use UnslothAI's LoRA optimization for 2x faster generation
))

messages = []
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break

  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})     # add query to messages
  print("Assistant: ", end="", flush=True)
  response = ""
  for new_text in chat_model.stream_chat(messages):      # stream generation
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response}) # add response to messages

torch_gc()

[INFO|tokenization_utils_base.py:2087] 2024-04-24 18:56:03,682 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/tokenizer.json
[INFO|tokenization_utils_base.py:2087] 2024-04-24 18:56:03,683 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2087] 2024-04-24 18:56:03,685 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/special_tokens_map.json
[INFO|tokenization_utils_base.py:2087] 2024-04-24 18:56:03,687 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/tokenizer_config.json


/content/LLaMA-Factory


[WARNING|logging.py:314] 2024-04-24 18:56:04,105 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


04/24/2024 18:56:04 - INFO - llmtuner.data.template - Replace eos token: <|eot_id|>


INFO:llmtuner.data.template:Replace eos token: <|eot_id|>
[INFO|configuration_utils.py:726] 2024-04-24 18:56:04,211 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/config.json
[INFO|configuration_utils.py:789] 2024-04-24 18:56:04,214 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "b

...

04/24/2024 18:56:25 - INFO - llmtuner.model.adapter - Loaded adapter(s): llama3_lora


INFO:llmtuner.model.adapter:Loaded adapter(s): llama3_lora


04/24/2024 18:56:25 - INFO - llmtuner.model.loader - all params: 8051232768


INFO:llmtuner.model.loader:all params: 8051232768



User: who are you
Assistant: I am Llama-3, an AI assistant developed by LLaMA Factory.

User: give me 3 tips to keep healthy
Assistant: 1. Stay hydrated: Drinking plenty of water throughout the day helps to flush out toxins, maintain energy levels, and keep your skin looking healthy and clear.

2. Eat a balanced diet: A well-rounded diet that includes a variety of fruits, vegetables, whole grains, and lean protein sources can help to support overall health and well-being.

3. Get regular exercise: Engaging in regular physical activity can help to boost mood, improve sleep, and increase energy levels. Aim for at least 30 minutes of moderate exercise per day.

User: thanks
Assistant: You're welcome!

User: exit
